In [1]:
import numpy
import pywt
from matplotlib import pyplot
from PIL import Image
from skimage.feature import greycomatrix

In [ ]:
from toolbox.classification.parameters import Dermatology
inputs = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=False)
benin = inputs['Datum'][4884]
malin = inputs['Datum'][323]

In [ ]:
# Calcul FFT
benin_fft = numpy.fft.fft2(benin)
benin_fft = numpy.fft.fftshift(benin_fft)
malin_fft = numpy.fft.fft2(malin)
malin_fft = numpy.fft.fftshift(malin_fft)
fft_titles = ['Module', 'Phase']

images = ['Benin', 'Malin']
directions = ['Horizontal', 'Diagonale_1', 'Vertical', 'Diagonale_2']

fig = pyplot.figure()
fig, axes = pyplot.subplots(2, 3, figsize=(8, 4), dpi=600)

axes[0, 0].imshow(benin, cmap='gray')
axes[0, 0].axis('off')
axes[0, 1].imshow(numpy.log(1+numpy.abs(benin_fft)), cmap='gray')
axes[0, 1].set_title('Module')
axes[0, 1].axis('off')
axes[0, 2].imshow(numpy.angle(benin_fft), cmap='gray')
axes[0, 2].set_title('Phase')
axes[0, 2].axis('off')
axes[1, 0].imshow(malin, cmap='gray')
axes[1, 0].axis('off')
axes[1, 1].imshow(numpy.log(1+numpy.abs(malin_fft)), cmap='gray')
axes[1, 1].axis('off')
axes[1, 2].imshow(numpy.angle(malin_fft), cmap='gray')
axes[1, 2].axis('off')

pyplot.show()
fig.savefig('example_fft.pdf', bbox_inches='tight')

benin = benin.astype(numpy.float32)
malin = malin.astype(numpy.float32)

max_lev = 3       # how many levels of decomposition to draw
label_levels = 3  # how many levels to explicitly label on the plots

fig, axes = pyplot.subplots(2, 4, figsize=[14, 8], dpi=600)
for level in range(0, max_lev + 1):
    # compute the 2D DWT
    c = pywt.wavedec2(benin, 'haar', level=level)
    # normalize each coefficient array independently for better visibility
    c[0] /= numpy.abs(c[0]).max()
    for detail_level in range(level):
        c[detail_level + 1] = [d/numpy.abs(d).max() for d in c[detail_level + 1]]
    # show the normalized coefficients
    arr, slices = pywt.coeffs_to_array(c)
    axes[0, level].imshow(arr, cmap='gray')
    axes[0, level].set_title('Coefficients\n({} niveaux)'.format(level))
    axes[0, level].set_axis_off()

for level in range(0, max_lev + 1):
    # compute the 2D DWT
    c = pywt.wavedec2(malin, 'haar', level=level)
    # normalize each coefficient array independently for better visibility
    c[0] /= numpy.abs(c[0]).max()
    for detail_level in range(level):
        c[detail_level + 1] = [d/numpy.abs(d).max() for d in c[detail_level + 1]]
    # show the normalized coefficients
    arr, slices = pywt.coeffs_to_array(c)
    axes[1, level].imshow(arr, cmap='gray')
    # axes[1, level].set_title('Coefficients\n({} level)'.format(level))
    axes[1, level].set_axis_off()

pyplot.show()
fig.savefig('example_wavelet.pdf', bbox_inches='tight')

In [ ]:
originales = [benin, malin]

benin_glcm = greycomatrix(benin, distances=[1], angles=[0, numpy.pi/4.0, numpy.pi/2.0, 3*numpy.pi/4.0], levels=256, symmetric=True, normed=False)
malin_glcm = greycomatrix(malin, distances=[1], angles=[0, numpy.pi/4.0, numpy.pi/2.0, 3*numpy.pi/4.0], levels=256, symmetric=True, normed=False)
glcm = [benin_glcm, malin_glcm]

images = ['Benin', 'Malin']
directions = ['Horizontal', 'Diagonale_1', 'Vertical', 'Diagonale_2']

fig = pyplot.figure()
fig, axes = pyplot.subplots(2, 5, figsize=(8, 4), dpi=1200)
for im_index, image in enumerate(images):
    axes[im_index, 0].imshow(originales[im_index], cmap='gray', vmin=0, vmax=255)
    axes[im_index, 0].axis('off')
    for index, direction in enumerate(directions):
        data = glcm[im_index][:,:,0,index]
        im = axes[im_index, index+1].imshow(data, cmap='gray', vmin=0, vmax=2500)
        axes[im_index, index+1].set_title(f'{direction}')
        axes[im_index, index+1].axis('off')
fig.subplots_adjust(right=0.8)
pyplot.axis('off')
pyplot.show()

fig.savefig('example_GLCM.pdf', bbox_inches='tight')